In [22]:
import fastbook
from fastbook import * 
from fastai.vision.all import *

In [23]:
path = untar_data(URLs.MNIST_SAMPLE)
threes = (path/'train'/'3')
sevens = (path/'train'/'7')

In [24]:
stacked_threes = torch.stack([tensor(Image.open(o)) for o in threes.ls()]).float().view(-1, 28*28)
stacked_sevens = torch.stack([tensor(Image.open(o)) for o in sevens.ls()]).float().view(-1, 28*28)
# Label below
stacked_labels = torch.cat([torch.zeros(len(stacked_threes)), torch.ones(len(stacked_sevens))])
stacked_threes.shape, stacked_sevens.shape, stacked_labels.shape

(torch.Size([6131, 784]), torch.Size([6265, 784]), torch.Size([12396]))

In [25]:
# create a dataset and dataloader for our epoch
dataset = list(zip(torch.cat([stacked_threes, stacked_sevens]), stacked_labels))
dl = DataLoader(dataset, batch_size=64, shuffle=True) 

In [26]:
def batch_accuracy(preds, targs):
    preds = preds.sigmoid()
    return (preds > 0.5).eq(targs).float().mean()

In [41]:
def validate_epoch(model):
    accs = [batch_accuracy(model(x),y) for x,y in dl]
    return round(torch.stack(accs).mean().item(),4)

In [28]:
linear_model = nn.Linear(28*28, 1)

In [29]:
w,b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [30]:
class BasicOptimizer:
    def __init__(self, params, lr=0.01):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            p.data -= self.lr * p.grad.data

    def zero_grad(self):
        for p in self.params:
            p.grad = None

In [31]:
opt = BasicOptimizer(linear_model.parameters(), lr=0.01)

In [32]:
def mnist_loss(preds, targs):
    preds_sigmoid = preds.sigmoid()
    return torch.where(targs == 1, 1-preds_sigmoid,  preds_sigmoid).mean()

In [33]:
def calc_grad(x, y, model):
    preds = model(x)
    loss = mnist_loss(preds, y)
    loss.backward()
    return loss

In [34]:
# Start the epoch for the calculation
def train_epoch(dl, model, opt):
    for x, y in dl:
        calc_grad(x, y, model)
        opt.step()
        opt.zero_grad()

        

In [52]:
def train_model(model,  epochs=1):
    opt1 = BasicOptimizer(linear_model.parameters(), lr=0.01)

    for epoch in range(epochs):
        train_epoch(dl, model, opt1)
        acc = validate_epoch(model)
        print(f'Epoch {epoch+1}/{epochs}, Accuracy: {acc}')

In [53]:
train_model(linear_model, epochs=1000)

Epoch 1/1000, Accuracy: 0.5054
Epoch 2/1000, Accuracy: 0.5056
Epoch 3/1000, Accuracy: 0.5054
Epoch 4/1000, Accuracy: 0.5053
Epoch 5/1000, Accuracy: 0.5056
Epoch 6/1000, Accuracy: 0.5055
Epoch 7/1000, Accuracy: 0.5056
Epoch 8/1000, Accuracy: 0.5056
Epoch 9/1000, Accuracy: 0.5053
Epoch 10/1000, Accuracy: 0.5054
Epoch 11/1000, Accuracy: 0.5054
Epoch 12/1000, Accuracy: 0.5053
Epoch 13/1000, Accuracy: 0.5053
Epoch 14/1000, Accuracy: 0.5054
Epoch 15/1000, Accuracy: 0.5054
Epoch 16/1000, Accuracy: 0.5053
Epoch 17/1000, Accuracy: 0.5054
Epoch 18/1000, Accuracy: 0.5055
Epoch 19/1000, Accuracy: 0.5054
Epoch 20/1000, Accuracy: 0.5052
Epoch 21/1000, Accuracy: 0.5054
Epoch 22/1000, Accuracy: 0.5053
Epoch 23/1000, Accuracy: 0.5055
Epoch 24/1000, Accuracy: 0.5052
Epoch 25/1000, Accuracy: 0.5053
Epoch 26/1000, Accuracy: 0.5052
Epoch 27/1000, Accuracy: 0.5053
Epoch 28/1000, Accuracy: 0.5053
Epoch 29/1000, Accuracy: 0.5055
Epoch 30/1000, Accuracy: 0.5055
Epoch 31/1000, Accuracy: 0.5052
Epoch 32/1000, Ac

KeyboardInterrupt: 